<a href="https://colab.research.google.com/github/ViniciusTavaresSousa/3D-DCGAN/blob/main/3D_DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**3D DCGAN**

In [4]:
import tensorflow as tf
from tensorflow.keras import layers
import os
import numpy as np
import time

##**Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##**Base de Dados**

In [5]:
voxels_treinamento = np.load("/content/drive/MyDrive/3D DCGAN/airplane.npy", allow_pickle = True)

In [6]:
voxels_treinamento.shape

(451, 64, 64, 64)

In [7]:
buffer_size = 451
batch_size = 16

In [8]:
# Cria um conjunto de dados a partir dos voxels, embaralha e divide em lotes
base_de_dados = tf.data.Dataset.from_tensor_slices(voxels_treinamento).shuffle(buffer_size).batch(batch_size)

##**Gerador**

In [ ]:
def criaGerador():
  modelo = tf.keras.Sequential()

  modelo.add(layers.Dense(4*4*4*512, use_bias = False, input_shape = (150,)))
  modelo.add(layers.BatchNormalization())
  modelo.add(layers.LeakyReLU())

  modelo.add(layers.Reshape((4, 4, 4, 512)))

  modelo.add(layers.Conv3DTranspose(256, (4, 4, 4), strides = (2, 2, 2), padding = 'same', use_bias = False))
  modelo.add(layers.BatchNormalization())
  modelo.add(layers.LeakyReLU())

  modelo.add(layers.Conv3DTranspose(128, (4, 4, 4), strides = (2, 2, 2), padding = 'same', use_bias = False))
  modelo.add(layers.BatchNormalization())
  modelo.add(layers.LeakyReLU())

  modelo.add(layers.Conv3DTranspose(64, (4, 4, 4), strides = (2, 2, 2), padding = 'same', use_bias = False))
  modelo.add(layers.BatchNormalization())
  modelo.add(layers.LeakyReLU())

  modelo.add(layers.Conv3DTranspose(1, (4, 4, 4), strides = (2, 2, 2), padding = 'same', use_bias = False, activation = 'sigmoid'))

  modelo.summary()

  return modelo

In [ ]:
gerador = criaGerador()

##**Discriminador**

In [ ]:
def criaDiscriminador():
  modelo = tf.keras.Sequential()

  modelo.add(layers.Conv3D(16, (4, 4, 4), strides = (2, 2, 2), padding = 'same', input_shape = [64, 64, 64, 1]))
  modelo.add(layers.LeakyReLU())
  modelo.add(layers.Dropout(0.3))


  modelo.add(layers.Conv3D(32, (4, 4, 4), strides = (2, 2, 2), padding = 'same'))
  modelo.add(layers.LeakyReLU())
  modelo.add(layers.Dropout(0.3))


  modelo.add(layers.Conv3D(64, (4, 4, 4), strides = (2, 2, 2), padding = 'same'))
  modelo.add(layers.LeakyReLU())
  modelo.add(layers.Dropout(0.3))

  modelo.add(layers.Conv3D(128, (4, 4, 4), strides = (2, 2, 2), padding = 'same'))
  modelo.add(layers.LeakyReLU())
  modelo.add(layers.Dropout(0.3))

  modelo.add(layers.Flatten())
  modelo.add(layers.Dense(1))

  modelo.summary()

  return modelo

In [ ]:
discriminador = criaDiscriminador()

##**Função de Perda**

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits = True)

In [ ]:
def perdaDiscriminador(saida_real, saida_falsa):

  saida_real = cross_entropy(tf.ones_like(saida_real), saida_real)
  saida_falsa = cross_entropy(tf.zeros_like(saida_falsa), saida_falsa)
  perda_total = saida_real + saida_falsa

  return perda_total

In [ ]:
def perdaGerador(saida_falsa):

  return cross_entropy(tf.ones_like(saida_falsa), saida_falsa)

In [ ]:
# Inicializa os otimizadores
otimizador_gerador = tf.keras.optimizers.Adam(learning_rate = 1e-4)
otimizador_discriminador = tf.keras.optimizers.Adam(learning_rate = 1e-4)

In [ ]:
# Diretório para checkpoints do treinamento
checkpoint_dir = "/content/drive/MyDrive/3D DCGAN"
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(otimizador_gerador = otimizador_gerador,
                                 otimizador_discriminador= otimizador_discriminador,
                                 gerador = gerador,
                                 discriminador = discriminador)

##**Treinamento**

In [ ]:
epocas = 1630 # Número de épocas de treinamento
dim_ruido = 150 # Dimensão do vetor de ruído
num_de_exemplos_gerados = 1 # Número de exemplos a gerar

In [ ]:
# Diretório para armazenar os logs do TensorBoard
log_dir = "/content/drive/MyDrive/3D DCGAN/logs/gan/" + str(batch_size) + "_" + str(dim_ruido) + "_" + str(epocas)
summary_writer = tf.summary.create_file_writer(log_dir)

In [ ]:
# Vetor latente para geração de voxels
seed = tf.random.normal([num_de_exemplos_gerados, dim_ruido])

In [ ]:
@tf.function
def passos_treinamento(voxels, epoca):
    ruido = tf.random.normal([batch_size, dim_ruido])

    with tf.GradientTape() as tape_gerador, tf.GradientTape() as tape_discriminador:
        voxels_gerados = gerador(ruido, training=True)

        # Avalia as saídas do discriminador
        saida_real = discriminador(voxels, training=True)
        saida_falsa = discriminador(voxels_gerados, training=True)

        # Calcula a perda
        perda_ger = perdaGerador(saida_falsa)
        perda_disc = perdaDiscriminador(saida_real, saida_falsa)

    # Calcula os gradientes
    gradientes_gerador = tape_gerador.gradient(perda_ger, gerador.trainable_variables)
    gradientes_discriminador = tape_discriminador.gradient(perda_disc, discriminador.trainable_variables)

    # Aplica os gradientes
    otimizador_gerador.apply_gradients(zip(gradientes_gerador, gerador.trainable_variables))
    otimizador_discriminador.apply_gradients(zip(gradientes_discriminador, discriminador.trainable_variables))

    # Registra as perdas no TensorBoard
    with summary_writer.as_default():
        tf.summary.scalar('Perda do Gerador', perda_ger, step=epoca)
        tf.summary.scalar('Perda do Discriminador', perda_disc, step=epoca)


In [ ]:
def gerar_e_salvar_voxels(modelo, epoca, entrada_teste):
    # Gera os voxels
    predicoes = modelo(entrada_teste, training=False)

    # Salva cada voxel gerado
    for i in range(predicoes.shape[0]):
        voxel = predicoes[i].numpy()
        np.save(f"/content/drive/MyDrive/3D DCGAN/voxels/voxel_{epoca:04d}_{i}.npy", voxel)

    print(f'Voxels salvos para a época {epoca}')

In [ ]:
def treinar(conjunto_dados, epocas):
    for epoca in range(epocas):
        inicio = time.time()
        for lote_voxels in conjunto_dados:
            passos_treinamento(lote_voxels, epoca)

        gerar_e_salvar_voxels(gerador, epoca + 1, seed)

        if (epoca + 1) % 15 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

        print(f"Tempo para a época {epoca + 1} em {time.time() - inicio:.2f} segundos")

    gerar_e_salvar_voxels(gerador, epoca + 1, seed)

In [ ]:
# Carrega o TensorBoard
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/3D\ DCGAN/logs/gan/

In [ ]:
# Inicia o treinamento
treinar(base_de_dados, epocas)